In [ ]:
!pip3.9 install requests pandas scikit-learn matplotlib python-dotenv

In [17]:
import requests
from datetime import timedelta, date
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
import os

load_dotenv()  

True

# Data Collection

In [18]:
year_data = pd.DataFrame()
year = 2021

URL="https://www.rescuetime.com/anapi/data"
params = {
"key": os.getenv("RESCUETIME_KEY"),
"perspective": "interval",
"interval": "hour",
"format": "csv",
}

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)


def generate_data(year): 
    year_data =  pd.DataFrame()

    start_date = date(year, 1, 1)
    end_date = date(year, 12, 31)
    
    for single_date in daterange(start_date, end_date):

        params["restrict_begin"] = single_date.strftime("%Y-%m-%d")
        params["restrict_end"] = single_date.strftime("%Y-%m-%d")
        
        response = requests.get(URL, params=params)
        
        if response.status_code == 200:
            # Create a StringIO object from the response text and read it into a DataFrame
            data = pd.read_csv(StringIO(response.text))
            # Append the data to the main DataFrame
            year_data = pd.concat([year_data,data], ignore_index=True)
            print(f"Added {single_date.strftime('%Y-%m-%d')}")
        else:
            print(f"Request for {single_date.strftime('%Y-%m-%d')} failed with status code {response.status_code}")
    
    return year_data



In [ ]:
years = [2020,2021,2022]
for year in years:
    year_data = generate_data(year)
    year_data.to_csv(f'{year}.csv', index=False)

In [ ]:
year_data.to_csv(f'{year}.csv', index=False)